In [115]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
# Pkg.add("BlockDiagonals")
# Pkg.add("TrajOptPlots")
using OSQP
using Plots
using SparseArrays
using LinearAlgebra
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor
using RobotZoo
using RobotDynamics
using ForwardDiff
using BlockDiagonals
using ControlSystems

  Activating environment at `D:\Study_material\Grad_Courses\16745\HW2\homework2-Wenyu-Wu\Project.toml`


In [2]:
#Quaternion stuff
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I]
function qtoQ(q)
    return H'*T*L(q)*T*L(q)*H
end
function G(q)
    G = L(q)*H
end
function rptoq(ϕ)
    (1/sqrt(1+ϕ'*ϕ))*[1; ϕ]
end
function qtorp(q)
    q[2:4]/q[1]
end

qtorp (generic function with 1 method)

In [3]:
#Quadrotor parameters
mass = 0.5
ℓ = 0.1750
J = Diagonal([0.0023, 0.0023, 0.004])
g = 9.81
kt=1.0
km=0.0245

h = 0.05 #20 Hz

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4     # number of controls
Tfinal = 5.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

In [4]:
model = Quadrotor()
const n = Nx̃
# const n = Nx
const m = Nu
const dt = 0.1  # time step (s)
const tf = 25   # time horizon (s)
# const N = Int(tf / dt) + 1;

In [5]:
function E(q)
    E = BlockDiagonal([1.0*I(3), G(q), 1.0*I(6)])
end

E (generic function with 1 method)

In [6]:
function quad_dynamics(x,u)
    r = x[1:3]
    q = x[4:7]
    v = x[8:10]
    ω = x[11:13]
    Q = qtoQ(q)
    
    ṙ = Q*v
    q̇ = 0.5*L(q)*H*ω
    
    v̇ = Q'*[0; 0; -g] + (1/mass)*[zeros(2,4); kt*ones(1,4)]*u - hat(ω)*v
    
    ω̇ = J\(-hat(ω)*J*ω + [0 ℓ*kt 0 -ℓ*kt; -ℓ*kt 0 ℓ*kt 0; km -km km -km]*u)
    
    return [ṙ; q̇; v̇; ω̇]
end

quad_dynamics (generic function with 1 method)

In [7]:
function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[4:7] .= xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
    return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [8]:
#Initial Conditions
uhover = (mass*g/4)*ones(4)
r0 = [0.0; 0; 1.0]
q0 = [1.0; 0; 0; 0]
v0 = zeros(3)
ω0 = zeros(3)
x0 = [r0; q0; v0; ω0];
# linearize around hover state
xeq = x0
ueq = uhover;

In [9]:
#Linearize dynamics about hover
A = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),x0)
B = ForwardDiff.jacobian(u->quad_dynamics_rk4(x0,u),uhover);

In [10]:
#Reduced system
Ã = Array(E(q0)'*A*E(q0))
B̃ = Array(E(q0)'*B);

In [11]:
# Cost weights
Q = Array(I(Nx̃));
R = Array(.1*I(Nu));

In [12]:
function nominal_trajectory(x0,N,dt)
    Xref = [fill(0.0, length(x0)) for k = 1:N]
    dx = x0[1]/(N-1)
    dy = x0[2]/(N-1)
    dz = x0[3]/(N-1)
    # TODO: Design a trajectory that linearly interpolates from x0 to the origin
    for i = 1:length(Xref)
        Xref[i][1] = x0[1] - (dx*(i-1))
        Xref[i][2] = x0[2] - (dy*(i-1))
        Xref[i][3] = x0[3] - (dz*(i-1))
        Xref[i][4] = 1.0
        Xref[i][8] = -dx/dt
        Xref[i][9] = -dy/dt
        Xref[i][10] = -dz/dt
    end
    Xref[end][8] = 0.0
    Xref[end][9] = 0.0
    Xref[end][10] = 0.0
#     return SVector{8}.(Xref)
    return Xref
end

nominal_trajectory (generic function with 1 method)

In [50]:
function report_trajectory(x0,N,dt)
    xorigin = [0.0,0.0,-1.0]
    Xref = [fill(0.0, length(x0)) for k = 1:N]
    dx = x0[1]/(N-1)
    dy = x0[2]/(N-1)
    dz = (x0[3]-xorigin[3])/(N-1)
    # TODO: Design a trajectory that linearly interpolates from x0 to the origin
    for i = 1:length(Xref)
        Xref[i][1] = x0[1] - (dx*(i-1))
        Xref[i][2] = x0[2] - (dy*(i-1))
        Xref[i][3] = x0[3] - (dz*(i-1))
        Xref[i][4] = 1.0
        Xref[i][8] = -dx/dt
        Xref[i][9] = -dy/dt
        Xref[i][10] = -dz/dt
    end
    Xref[1][8:10] .= 0.0
    Xref[end][8] = 0.0
    Xref[end][9] = 0.0
    Xref[end][10] = 0.0
#     return SVector{8}.(Xref)
    return Xref
end

report_trajectory (generic function with 1 method)

In [127]:
unit_ref = zeros(size(x0))
unit_ref[1:4] .= 1.0
x0_ref = unit_ref + x0
Xref = nominal_trajectory(x0_ref,Nt,h)
Uref = [copy(uhover) for k = 1:Nt]
tref = range(0,Tfinal,length=Nt);
Xref[end][8:10] .= 0.0
Xref[1][8:10] .= 0.0

Xreport = report_trajectory(x0_ref,Nt,h)
Ureport = [copy(uhover) for k = 1:Nt]
println(Xref[end])
println(Xreport[end])

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, -1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [14]:
struct LQRController
    K::Matrix{Float64}
    Xref::Vector{Vector{Float64}}
    Uref::Vector{Vector{Float64}}
    times::Vector{Float64}
end

"""
    get_k(ctrl, t)

Get the time index corresponding to time `t`. 
Useful for implementing zero-order hold control.
Uses binary search to find the time index.
"""
get_k(controller, t) = searchsortedlast(controller.times, t)

"""
    get_control(ctrl, x, t)

Evaluate the LQR feedback policy at state `x` and time `t`, returning the control 
to be executed by the system.
"""
function get_control(ctrl::LQRController, x, t)
    k = get_k(ctrl, t)
    x0 = ctrl.Xref[k]
    # TODO: Implement the control policy
    # SOLUTION
    q0 = x0[4:7]
    q = x[4:7]
    ϕ = qtorp(L(q0)'*q)
    r0 = x0[1:3]
    v0 = x0[8:10]
    ω0 = x0[11:13]
    Δx = [x[1:3]-r0; ϕ; x[8:10]-v0; x[11:13]-ω0]
#     u = ctrl.Uref[k] - ctrl.K*(x - ctrl.Xref[k])
    u = ctrl.Uref[k] - ctrl.K*Δx
    return u
end

"""
    lqr(A,B,Q,R; kwargs...)

Calculate the infinite-horizon LQR gain from dynamics Jacobians `A` and `B` and
cost matrices `Q` and `R`. Returns the infinite-horizon gain `K` and cost-to-go `P`.

# Keyword Arguments
* `P`: Provide an initial guess for the infinite-horizon gain
* `max_iters`: maximum number of iterations
* `tol`: tolerance for solve
` `verbose`: print the number of iterations
"""
function lqr(A,B,Q,R; P=Matrix(Q), tol=1e-8, max_iters=400, verbose=false)
    # initialize the output
    n,m = size(B)
    K = zeros(m,n)
    
    # TODO: calculate the infinite-horizon LQR solution

    P_vec = [zeros(n,n) for k = 1:max_iters]
    K_vec = [zeros(m,n) for k = 1:max_iters-1]
    P_vec[end] = Q
    for k = reverse(1:max_iters-1) 
        K_vec[k] .= (R + B'P_vec[k+1]*B)\(B'P_vec[k+1]*A)
        P_vec[k] .= Q + A'P_vec[k+1]*A - A'P_vec[k+1]*B*K_vec[k]
    end
    
    # return the feedback gains and ctg matrices
    return K_vec[1],P_vec[1]
end

lqr

In [128]:
K,Qf = lqr(Ã,B̃,Q,R)
# ctrl = LQRController(K,Xref,Uref,tref);
ctrl = LQRController(K,Xreport,Ureport,tref);
rinit = Xref[1][1:3]
qinit = Xref[1][4:7]
vinit = Xref[1][8:10]
ωinit = Xref[1][11:13]
xinit = [rinit+randn(3); L(qinit)*rptoq(0.7*randn(3)); vinit; ωinit];
println(xinit)

[-1.0324529781524001, 1.5942098985964621, 1.0366424392495, 0.9110143447952838, -0.18176357822921868, 0.23236539137176793, -0.2881339794248078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [91]:
lqr_error_ls = [];

In [129]:
times = thist
N = length(thist)
Xlqr = [zeros(n) for k = 1:N] 
Ulqr = [zeros(m) for k = 1:N-1]
Xlqr[1] = xinit

for k = 1:N-1
    Ulqr[k] = get_control(ctrl, Xlqr[k], times[k])
    Xlqr[k+1] = quad_dynamics_rk4(Xlqr[k],Ulqr[k])
end
tlqr = times;
lqr_error = norm(Xlqr[end][1:3]-Xref[end][1:3])
append!(lqr_error_ls,lqr_error)

println(Xlqr[end])
println(lqr_error)
println(lqr_error_ls)
# println(Ulqr[15])

[-0.05080629190807467, 0.0008391626863310919, -1.0060342915534655, 0.9995697577839568, 0.001190284234571408, -0.0017351464509727979, -0.029255286940251295, -0.1551895295849367, -0.19233776005275396, -0.5941588343960836, -0.0017017035001666732, 0.003945162245022656, 0.029271192493075724]
1.0073167224229655
Any[0.048129698940060126, 0.14467892099703453, 0.0747515628887346, 0.05385364396996052, 0.08707828982429716, 1.0073167224229655]


In [17]:
#Set up visualization
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor

vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat C:\Users\ASUS\.julia\packages\MeshCat\GlCMx\src\visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

In [54]:
model = Quadrotor()
TrajOptPlots.set_mesh!(vis, model)
Xlqr = SVector{13}.(Xlqr)
visualize!(vis, model, tlqr[end], Xlqr)

In [19]:
"""
    MPCController

An MPC controller that uses a solver of type `S` to solve a QP at every iteration.

It will track the reference trajectory specified by `Xref`, `Uref` and `times` 
with an MPC horizon of `Nmpc`. It will track the terminal reference state if 
the horizon extends beyond the reference horizon.
"""
struct MPCController{S}
    P::SparseMatrixCSC{Float64,Int}
    q::Vector{Float64}
    A::SparseMatrixCSC{Float64,Int}
    lb::Vector{Float64}
    ub::Vector{Float64}
    Nmpc::Int
    solver::S
    Xref::Vector{Vector{Float64}}
    Uref::Vector{Vector{Float64}}
    times::Vector{Float64}
end

"""
    OSQPController(n,m,N,Nref,Nd)

Generate an `MPCController` that uses OSQP to solve the QP.
Initializes the controller with matrices consistent with `n` states,
`m` controls, and an MPC horizon of `N`, and `Nd` constraints. 

Use `Nref` to initialize a reference trajectory whose length may differ from the 
horizon length.
"""
function OSQPController(n::Integer, m::Integer, N::Integer, Nref::Integer=N, Nd::Integer=(N-1)*n)
    Np = (N-1)*(n+m)   # number of primals
    P = spzeros(Np,Np)
    q = zeros(Np)
    A = spzeros(Nd,Np)
    lb = zeros(Nd)
    ub = zeros(Nd)
    Xref = [zeros(n) for k = 1:Nref]
    Uref = [zeros(m) for k = 1:Nref]
    tref = zeros(Nref)
    solver = OSQP.Model()
    MPCController{OSQP.Model}(P,q, A,lb,ub, N, solver, Xref, Uref, tref)
end

# isconstrained(ctrl::MPCController) = length(ctrl.lb) != (ctrl.Nmpc - 1) * length(ctrl.Xref[1])
isconstrained(ctrl::MPCController) = length(ctrl.lb) != (ctrl.Nmpc - 1) * 12

"""
    buildQP!(ctrl, A,B,Q,R,Qf; kwargs...)

Build the QP matrices `P` and `A` for the MPC problem. Note that these matrices
should be constant between MPC iterations.

Any keyword arguments will be passed to `initialize_solver!`.
"""
function buildQP!(ctrl::MPCController, A,B,Q,R,Qf; kwargs...)
    if isconstrained(ctrl)
        buildQP_constrained!(ctrl::MPCController, A,B,Q,R,Qf; kwargs...)
    else
        buildQP_unconstrained!(ctrl::MPCController, A,B,Q,R,Qf; kwargs...)
    end
end

"""
    updateQP!(ctrl::MPCController, x, time)

Update the vectors in the QP problem for the current state `x` and time `time`.
This should update `ctrl.q`, `ctrl.lb`, and `ctrl.ub`.
"""
function updateQP!(ctrl::MPCController, x, time)
    if isconstrained(ctrl)
        updateQP_constrained!(ctrl, x, time)
    else
        updateQP_unconstrained!(ctrl, x, time)
    end
end


"""
    initialize_solver!(ctrl::MPCController; kwargs...)

Initialize the internal solver once the QP matrices are initialized in the 
controller.
"""
function initialize_solver!(ctrl::MPCController{OSQP.Model}; tol=1e-6, verbose=false)
    OSQP.setup!(ctrl.solver, P=ctrl.P, q=ctrl.q, A=ctrl.A, l=ctrl.lb, u=ctrl.ub, 
        verbose=verbose, eps_rel=tol, eps_abs=tol, polish=1)
end

"""
    get_control(ctrl::MPCController, x, t)

Get the control from the MPC solver by solving the QP. 
If you want to use your own QP solver, you'll need to change this
method.
"""
function get_control(ctrl::MPCController{OSQP.Model}, x, time)
    # Update the QP
    updateQP!(ctrl, x, time)
    OSQP.update!(ctrl.solver, q=ctrl.q, l=ctrl.lb, u=ctrl.ub)

    # Solve QP
    results = OSQP.solve!(ctrl.solver)
#     Δu = results.x[1:2]
    Δu = results.x[1:4]
    
    k = get_k(ctrl, time)
    return ctrl.Uref[k] + Δu 
end

get_control

In [20]:
function buildQP_unconstrained!(ctrl::MPCController, A,B,Q,R,Qf; kwargs...)
     # TODO: Implement this method to build the QP matrices
    
    # SOLUTION:
    Nt = ctrl.Nmpc-1
#     Nx = length(ctrl.Xref[1])    # number of states
    Nx = 12
    Nu = length(ctrl.Uref[1])    # number of controls
    
    H = sparse([kron(Diagonal(I,Nt-1),[R zeros(Nu,Nx); zeros(Nx,Nu) Q]) zeros((Nx+Nu)*(Nt-1), Nx+Nu); zeros(Nx+Nu,(Nx+Nu)*(Nt-1)) [R zeros(Nu,Nx); zeros(Nx,Nu) Qf]])
    C = sparse([
            [B -I zeros(Nx,(Nt-1)*(Nu+Nx))]; 
            zeros(Nx*(Nt-1),Nu) [kron(Diagonal(I,Nt-1), [A B]) zeros((Nt-1)*Nx,Nx)] + [zeros((Nt-1)*Nx,Nx) kron(Diagonal(I,Nt-1),[zeros(Nx,Nu) Diagonal(-I,Nx)])]
    ])
    lb = zeros(Nx*Nt)
    ub = zeros(Nx*Nt)
    
    Nd = length(ctrl.lb)
    if Nd == Nt*n
        D = C
        ub = zero(ctrl.ub)
        lb = zero(ctrl.lb)
    end
    
    ctrl.P .= H
    ctrl.A .= C
    ctrl.ub .= ub
    ctrl.lb .= lb
    
    # Initialize the included solver
    #    If you want to use your QP solver, you should write your own
    #    method for this function
    initialize_solver!(ctrl; kwargs...)
    return nothing
end

# x diff helper function
function x_diff(x,x0)
    q0 = x0[4:7]
    q = x[4:7]
    ϕ = qtorp(L(q0)'*q)
    r0 = x0[1:3]
    v0 = x0[8:10]
    ω0 = x0[11:13]
    Δx = [x[1:3]-r0; ϕ; x[8:10]-v0; x[11:13]-ω0]
    return Δx
end


function updateQP_unconstrained!(ctrl::MPCController, x, time)
    # TODO: Implement this method
    
    # SOLUTION
    t = get_k(ctrl, time)
    
    Nt = ctrl.Nmpc-1             # horizon
#     Nx = length(ctrl.Xref[1])    # number of states
    Nx = 12
    Nu = length(ctrl.Uref[1])    # number of controls
    
    # Update QP problem
    b = ctrl.q
    lb = ctrl.lb
    ub = ctrl.ub
    xref = ctrl.Xref
#     xeq = Xref[end]
    xeq = x0
    N = length(ctrl.Xref)
    for t_h = 1:(Nt-1)
        if (t+t_h) <= N
            Δx = x_diff(xref[t+t_h],xeq)
#             b[(Nu+(t_h-1)*(Nx+Nu)).+(1:Nx)] .= -Q*(xref[t+t_h] - xeq)
            b[(Nu+(t_h-1)*(Nx+Nu)).+(1:Nx)] .= -Q*Δx
        else
            Δx = x_diff(xref[end],xeq)
#             b[(Nu+(t_h-1)*(Nx+Nu)).+(1:Nx)] .= -Q*(xref[end] - xeq)
            b[(Nu+(t_h-1)*(Nx+Nu)).+(1:Nx)] .= -Q*Δx
        end
    end
    if (t+Nt) <= N
        Δx = x_diff(xref[t+Nt],xeq)
#         b[(Nu+(Nt-1)*(Nx+Nu)).+(1:Nx)] .= -Qf*(xref[t+Nt] - xeq)
        b[(Nu+(Nt-1)*(Nx+Nu)).+(1:Nx)] .= -Qf*Δx
    else
        Δx = x_diff(xref[end],xeq)
#         b[(Nu+(Nt-1)*(Nx+Nu)).+(1:Nx)] .= -Qf*(xref[end] - xeq)
        b[(Nu+(Nt-1)*(Nx+Nu)).+(1:Nx)] .= -Qf*Δx
    end
    
    # Update the initial condition
#     lb[1:Nx] .= -A*(x - xeq)
    lb[1:Nx] .= -Ã*x_diff(x,xeq)
#     ub[1:Nx] .= -A*(x - xeq)
    ub[1:Nx] .= -Ã*x_diff(x,xeq)
    
    return nothing
end

updateQP_unconstrained! (generic function with 1 method)

In [130]:
# Initialize the unconstrained MPC problem
Nmpc = 51           # MPC Horizon
mpc1 = OSQPController(n, m, Nmpc, length(Xref))

# Provide the reference trajectory
# mpc1.Xref .= Xref
# mpc1.Uref .= Uref
mpc1.Xref .= Xreport
mpc1.Uref .= Ureport
mpc1.times .= tref

# Build the sparse QP matrices
buildQP!(mpc1, Ã,B̃,Q,R,Qf, tol=1e-2)

In [96]:
mpc_error_ls =[];

In [131]:
times = thist
N = length(thist)
Xmpc = [zeros(n) for k = 1:N] 
Umpc = [zeros(m) for k = 1:N-1]
Xmpc[1] = xinit
tmpc1 = times

for k = 1:N-1
    Umpc[k] = get_control(mpc1, Xmpc[k], times[k])
    Xmpc[k+1] = quad_dynamics_rk4(Xmpc[k],Umpc[k])
end
mpc_error = norm(Xmpc[end][1:3]-Xref[end][1:3])
append!(mpc_error_ls,mpc_error)
println(Xmpc[end])
println(mpc_error_ls)

[-0.02835072153667468, 0.01936777478740339, -0.9814872304026945, 0.9994939778404518, -0.009339221441221415, 0.009061189755561106, -0.02902519670486455, -0.04746175538312353, -0.08170980344128345, -0.21131526204265183, 0.0009480923343407853, -0.0010780036948591538, 0.029042974331561885]
Any[0.056569766703853684, 0.1327310799497157, 0.0579538928342695, 0.06970662736417468, 0.09447683941564115, 0.9820876017725799]


In [126]:
x_axis = 1:5
plot(x_axis,lqr_error_ls)
plot!(x_axis,mpc_error_ls)
plot!(title="error comparison")
xlabel!("random start point")
ylabel!("error")
savefig("error_comparison.png")
# println(x_axis)

In [135]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set(Any[WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED)]), ip"127.0.0.1", 8700))

In [136]:
Xmpc = SVector{13}.(Xmpc)
visualize!(vis, model, tmpc1[end] / 10, Xmpc)

In [82]:
function buildQP_constrained!(ctrl::MPCController, A,B,Q,R,Qf; kwargs...)
    # TODO: Implement this method to build the QP matrices
    
    # SOLUTION:
    Nt = ctrl.Nmpc-1
#     Nx = length(ctrl.Xref[1])    # number of states
    Nx = 12
    Nu = length(ctrl.Uref[1])    # number of controls

#     max_roll = deg2rad(model.max_roll)
#     xeq = Xref[end]
    xeq = x0
#     aeq = xeq[7:8]  # actuators
    aeq = uhover
    
    H = sparse([kron(Diagonal(I,Nt-1),[R zeros(Nu,Nx); zeros(Nx,Nu) Q]) zeros((Nx+Nu)*(Nt-1), Nx+Nu); zeros(Nx+Nu,(Nx+Nu)*(Nt-1)) [R zeros(Nu,Nx); zeros(Nx,Nu) Qf]])
    b = zeros(Nt*(Nx+Nu))
    C = sparse([
            [B -I zeros(Nx,(Nt-1)*(Nu+Nx))]; 
            zeros(Nx*(Nt-1),Nu) [kron(Diagonal(I,Nt-1), [A B]) zeros((Nt-1)*Nx,Nx)] + [zeros((Nt-1)*Nx,Nx) kron(Diagonal(I,Nt-1),[zeros(Nx,Nu) Diagonal(-I,Nx)])]
    ])
#     Z = kron(Diagonal(I,Nt), [0 0 0 1 0 0 0 0 0 0]) #Matrix that picks out all x2 (height)
#     Θ = kron(Diagonal(I,Nt), [0 0 0 0 1 0 0 0 0 0]) #Matrix that picks out all x3 (θ)
    Z = kron(Diagonal(I,Nt), [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0])
#     Θ = kron(Diagonal(I,Nt), [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0])
    U = kron(Diagonal(I,Nt), [I zeros(Nu,Nx)]) #Matrix that picks out all u
#     D = [C; Z; Θ; U]
#     println(size(ctrl.A))
#     D = [C;Z;U]
    D = [C;Z]
#     println(size(D))
    
#     umin = RobotZoo.umin(model)
#     umax = RobotZoo.umax(model)
    umin = -2.5*ones(Nu)
    umax = 2.5*ones(Nu)
#     lb = [zeros(Nx*Nt); zeros(Nt); -Inf*ones(Nt); kron(ones(Nt),umin-aeq)]
#     lb = [zeros(Nx*Nt);zeros(Nt)]
    lb = [zeros(Nx*Nt);zeros(Nt)]
#     ub = [zeros(Nx*Nt); Inf*ones(Nt); Inf*ones(Nt); kron(ones(Nt),umax-aeq)]
    ub = [zeros(Nx*Nt);100*ones(Nt)]
    
    Nd = length(ctrl.lb)
    if Nd == Nt*n
        D = C
        ub = zero(ctrl.ub)
        lb = zero(ctrl.lb)
    end
    lb = lb[1:Nd]
    ub = ub[1:Nd]
    
    ctrl.P .= H
    ctrl.A .= D
    ctrl.ub .= ub
    ctrl.lb .= lb
    
    # Initialize the included solver
    #    If you want to use your QP solver, you should write your own
    #    method for this function
    initialize_solver!(ctrl; kwargs...)
    return nothing
end

"""
    update_QP!(ctrl::MPCController, x, time)

Update the vectors in the QP problem for the current state `x` and time `time`.
This should update `ctrl.q`, `ctrl.lb`, and `ctrl.ub`.
"""
function updateQP_constrained!(ctrl::MPCController, x, time)
    # TODO: Implement this method
    
    # SOLUTION
    t = get_k(ctrl, time)
    
    Nt = ctrl.Nmpc-1             # horizon
#     Nx = length(ctrl.Xref[1])    # number of states
    Nx = 12
    Nu = length(ctrl.Uref[1])    # number of controls
    
    # Update QP problem
    b = ctrl.q
    lb = ctrl.lb
    ub = ctrl.ub
    xref = ctrl.Xref
#     xeq = Xref[end]
    xeq = x0
    N = length(ctrl.Xref)
    for t_h = 1:(Nt-1)
        if (t+t_h) <= N
            Δx = x_diff(xref[t+t_h],xeq)
#             b[(Nu+(t_h-1)*(Nx+Nu)).+(1:Nx)] .= -Q*(xref[t+t_h] - xeq)
            b[(Nu+(t_h-1)*(Nx+Nu)).+(1:Nx)] .= -Q*Δx
        else
            Δx = x_diff(xref[end],xeq)
#             b[(Nu+(t_h-1)*(Nx+Nu)).+(1:Nx)] .= -Q*(xref[end] - xeq)
            b[(Nu+(t_h-1)*(Nx+Nu)).+(1:Nx)] .= -Q*Δx
        end
    end
    if (t+Nt) <= N
        Δx = x_diff(xref[t+Nt],xeq)
#         b[(Nu+(Nt-1)*(Nx+Nu)).+(1:Nx)] .= -Qf*(xref[t+Nt] - xeq)
        b[(Nu+(Nt-1)*(Nx+Nu)).+(1:Nx)] .= -Qf*Δx
    else
        Δx = x_diff(xref[end],xeq)
#         b[(Nu+(Nt-1)*(Nx+Nu)).+(1:Nx)] .= -Qf*(xref[end] - xeq)
        b[(Nu+(Nt-1)*(Nx+Nu)).+(1:Nx)] .= -Qf*Δx
    end
    
    # Update the initial condition
#     lb[1:Nx] .= -A*(x - xeq)
    lb[1:Nx] .= -Ã*x_diff(x,xeq)
#     ub[1:Nx] .= -A*(x - xeq)
    ub[1:Nx] .= -Ã*x_diff(x,xeq)

    return nothing
end

updateQP_constrained!

In [137]:
Nd = (Nmpc-1)*(n+1)
mpc2 = OSQPController(n, m, Nmpc, length(Xref), Nd)
# mpc2.Xref .= Xref
# mpc2.Uref .= Uref
mpc2.Xref .= Xreport
mpc2.Uref .= Ureport
mpc2.times .= tref
buildQP!(mpc2, Ã,B̃,Q,R,Qf, tol=1e-2, verbose=false);

In [138]:
times2 = thist
N = length(thist)
Xmpc2 = [zeros(n) for k = 1:N] 
Umpc2 = [zeros(m) for k = 1:N-1]
Xmpc2[1] = xinit
tmpc2 = times

for k = 1:N-1
    Umpc2[k] = get_control(mpc2, Xmpc2[k], times2[k])
    Xmpc2[k+1] = quad_dynamics_rk4(Xmpc2[k],Umpc2[k])
end
println(Xmpc2[end])

[-0.03734945070286743, 0.021540313500375274, 0.9995456165956681, 0.9994682259849553, -0.010423920326115785, 0.009954254256972314, -0.029249272703930528, -0.04650820389656724, -0.08998335961279733, 0.02370020101335866, 0.0025636029722450014, -0.0029456369862365193, 0.02926796497045501]


In [139]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set(Any[WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED)]), ip"127.0.0.1", 8700))

In [140]:
Xmpc2 = SVector{13}.(Xmpc2)
visualize!(vis, model, tmpc2[end] / 10, Xmpc2)